# Desafío 3- Grupo 8

# El Problema de la Asistencia a la Entrevista

El dataset pertenece al proceso de reclutamiento de varias industrias de la India

Nuestro objetivo es tratar de clasificar, dadas las características del dataset, si una persona irá a la entrevista de trabajo o no.

El Dataset contiene datos acerca de la industria, la ubicación, la posición a buscar, el sexo del candidato, donde vive actualmente el candidato, en que ciudad se ubicará el trabajo y de donde es originariamente el candidato. Además, los recruiters formularon preguntas cuya intención es obtener una respuesta Sí/No

##### Importamos las librerías necesarias

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import re

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import linear_model

In [3]:
df = pd.read_csv('Interview.csv')
df.head()

,Date of Interview,Client name,Industry,Location,Position to be closed,Nature of Skillset,Interview Type,Name(Cand ID),Gender,Candidate Current Location,...,Are you clear with the venue details and the landmark.,Has the call letter been shared,Expected Attendance,Observed Attendance,Marital Status,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27
0,13.02.2015,Hospira,Pharmaceuticals,Chennai,Production- Sterile,Routine,Scheduled Walkin,Candidate 1,Male,Chennai,...,Yes,Yes,Yes,No,Single,NaN,NaN,NaN,NaN,NaN
1,13.02.2015,Hospira,Pharmaceuticals,Chennai,Production- Sterile,Routine,Scheduled Walkin,Candidate 2,Male,Chennai,...,Yes,Yes,Yes,No,Single,NaN,NaN,NaN,NaN,NaN
2,13.02.2015,Hospira,Pharmaceuticals,Chennai,Production- Sterile,Routine,Scheduled Walkin,Candidate 3,Male,Chennai,...,NaN,NaN,Uncertain,No,Single,NaN,NaN,NaN,NaN,NaN
3,13.02.2015,Hospira,Pharmaceuticals,Chennai,Production- Sterile,Routine,Scheduled Walkin,Candidate 4,Male,Chennai,...,Yes,Yes,Uncertain,No,Single,NaN,NaN,NaN,NaN,NaN
4,13.02.2015,Hospira,Pharmaceuticals,Chennai,Production- Sterile,Routine,Scheduled Walkin,Candidate 5,Male,Chennai,...,Yes,Yes,Uncertain,No,Married,NaN,NaN,NaN,NaN,NaN


#### Vamos a borrar el índice 1233 que no contiene datos

In [4]:
df.drop(df.index[[1233]], inplace = True)

In [5]:
df.columns

Index([u'Date of Interview', u'Client name', u'Industry', u'Location',
       u'Position to be closed', u'Nature of Skillset', u'Interview Type',
       u'Name(Cand ID)', u'Gender', u'Candidate Current Location',
       u'Candidate Job Location', u'Interview Venue',
       u'Candidate Native location',
       u'Have you obtained the necessary permission to start at the required time',
       u'Hope there will be no unscheduled meetings',
       u'Can I Call you three hours before the interview and follow up on your attendance for the interview',
       u'Can I have an alternative number/ desk number. I assure you that I will not trouble you too much',
       u'Have you taken a printout of your updated resume. Have you read the JD and understood the same',
       u'Are you clear with the venue details and the landmark.',
       u'Has the call letter been shared', u'Expected Attendance',
       u'Observed Attendance', u'Marital Status', u'Unnamed: 23',
       u'Unnamed: 24', u'Unnamed: 2

#### Dropeamos las columnas date of interview y Name(Cand ID) y columnas que no contienen datos

In [6]:
df.drop(['Date of Interview','Name(Cand ID)','Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26','Unnamed: 27','Expected Attendance'], axis=1, inplace=True)

#### Ponemos los datos en minúscula para poder codificar

In [7]:
columns_to_lower = df.columns
for column in columns_to_lower:
    df[column] = df[column].astype(str).str.lower()

In [8]:
df.describe()

,Client name,Industry,Location,Position to be closed,Nature of Skillset,Interview Type,Gender,Candidate Current Location,Candidate Job Location,Interview Venue,Candidate Native location,Have you obtained the necessary permission to start at the required time,Hope there will be no unscheduled meetings,Can I Call you three hours before the interview and follow up on your attendance for the interview,Can I have an alternative number/ desk number. I assure you that I will not trouble you too much,Have you taken a printout of your updated resume. Have you read the JD and understood the same,Are you clear with the venue details and the landmark.,Has the call letter been shared,Observed Attendance,Marital Status
count,1233,1233,1233,1233,1233,1233,1233,1233,1233,1233,1233,1233,1233,1233,1233,1233,1233,1233,1233,1233
unique,15,7,9,7,82,6,2,8,7,7,46,6,6,5,5,6,5,9,4,2
top,standard chartered bank,bfsi,chennai,routine,java/j2ee/struts/hibernate,scheduled walk in,male,chennai,chennai,chennai,chennai,yes,yes,yes,yes,yes,yes,yes,yes,single
freq,904,949,841,1023,220,456,965,841,893,852,595,921,954,955,937,942,948,934,782,767


#### Estudiamos el campo Skills

In [9]:
skills = list(set(df["Nature of Skillset"]))
skills = [re.split(", |,|/ |/", i) for i in skills]
skills = [skill for line in skills for skill in line]
skills = sorted(list(set(skills)))
skills

['- sapbo',
 '10.00 am',
 '11.30 am',
 '12.30 pm',
 '9.00 am',
 '9.30 am',
 'accounting operations',
 'als testing',
 'aml',
 'analytical r & d',
 'analytical r&d',
 'automation testing java',
 'banking operations',
 'basesas program',
 'biosimilars',
 'biosimiliars',
 'biosimillar',
 'cdd',
 'cdd kyc',
 'core java',
 'cots',
 'cots developer',
 'dot net',
 'emea',
 'etl',
 'fresher',
 'generic drugs \xe2\x80\x93 ra',
 'global labelling',
 'hadoop',
 'hibernate',
 'informatica',
 'j2ee',
 'java',
 'java ',
 'java developer',
 'java j2ee',
 'java jsf',
 'java tech lead',
 'java-sas',
 'jsf',
 'kyc',
 'l & l',
 'lcm -manager',
 'lending & liability',
 'lending and liabilities',
 'lending&liablities',
 'licensing \xe2\x80\x93 ra',
 'manager',
 'ms exchange)',
 'oracle',
 'oracle plsql',
 'product control',
 'production',
 'production support - sccm',
 'publishing',
 'ra label',
 'ra publishing',
 'regulatory',
 'reporting',
 'routine',
 'sas',
 'sccm',
 'sccm \xe2\x80\x93 sharepoint',
 's

In [10]:
invalid_skills = ['10.00 am','11.30 am','12.30 pm','9.00 am','9.30 am']
df = df.loc[[ skill not in invalid_skills for skill in df["Nature of Skillset"]]]

In [11]:
df["Nature of Skillset"].value_counts().sort_index()

- sapbo, informatica                       4
accounting operations                     86
als testing                               15
aml/kyc/cdd                               84
analytical r & d                          13
analytical r&d                             3
automation testing java                    7
banking operations                        24
basesas program/ reporting                 1
biosimilars                                1
biosimiliars                               6
biosimillar                                3
cdd kyc                                   52
core java                                 17
cots                                       4
cots developer                            13
dot net                                    9
emea                                       6
etl                                        9
fresher                                   86
generic drugs – ra                         4
global labelling                           6
hadoop    

#### Vamos a empezar a limpiar la base de datos. Para eso reemplazamos en esta primer parte algunas de las respuestas que dijeron los participantes de las entrevistas para tomarlas como booleanos

In [12]:
df["Can I Call you three hours before the interview and follow up on your attendance for the interview"] = df["Can I Call you three hours before the interview and follow up on your attendance for the interview"].replace(["No Dont","Na"],["no",np.nan])
df["Have you taken a printout of your updated resume. Have you read the JD and understood the same"] = df["Have you taken a printout of your updated resume. Have you read the JD and understood the same"].replace(["No- will take it soon","Not yet","Not yet","Na","na"],["no","no","no",np.nan,np.nan])
df["Can I have an alternative number/ desk number. I assure you that I will not trouble you too much"] = df["Can I have an alternative number/ desk number. I assure you that I will not trouble you too much"].replace(["No I have only thi number","na","Na"],["no",np.nan,np.nan])
df["Are you clear with the venue details and the landmark."] = df["Are you clear with the venue details and the landmark."].replace(["No- I need to check","na","Na"],["no",np.nan,np.nan])
df["Has the call letter been shared"] = df["Has the call letter been shared"].replace(["Havent Checked", "need to check","Not sure","yet to check","na","Na"],["unsure","unsure","unsure","unsure",np.nan,np.nan])
df["Hope there will be no unscheduled meetings"] = df["Hope there will be no unscheduled meetings"].replace(["na","cant say"],[np.nan,"unsure"])
df["Have you obtained the necessary permission to start at the required time"] = df["Have you obtained the necessary permission to start at the required time"].replace(['Not yet', 'yet to confirm',"na"],["no","no",np.nan])
df = df.replace(['nan'],[np.nan])

#### Utilizamos un diccionario para corregir algunos campos particulares que presentaban problemas de repetición, codificación, etc.

In [13]:
col_names =  ['lookup_regex', 'extract_regex', 'column_to_lookup','column_to_complete','value_type','value_offset','extra_condition']
diccionario  = pd.DataFrame(columns = col_names)
diccionario.loc[len(diccionario)] = ['lending & liability|lending and liabilities|lending&liablities', '', 'Nature of Skillset', 'Nature of Skillset', 'string','l & l', {'data_column' : 'Nature of Skillset', 'value' : ['lending & liability','lending and liabilities','lending&liablities'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['tech lead- mednet|tech lead-mednet', '', 'Nature of Skillset', 'Nature of Skillset', 'string','tech lead/mednet', {'data_column' : 'Nature of Skillset', 'value' : ['tech lead- mednet','tech lead-mednet'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['biosimilars|biosimiliars', '', 'Nature of Skillset', 'Nature of Skillset', 'string','biosimillar', {'data_column' : 'Nature of Skillset', 'value' : ['biosimilars','biosimiliars'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['java j2ee|j2ee|java ,j2ee', '', 'Nature of Skillset', 'Nature of Skillset', 'string','java/j2ee', {'data_column' : 'Nature of Skillset', 'value' : ['java j2ee','j2ee','java ,j2ee'], 'condition_func': (lambda x,y: x.isin(y))}]
#diccionario.loc[len(diccionario)] = ['sccm- networking|sccm-\(network', '', 'Nature of Skillset', 'Nature of Skillset', 'string','sccm/networking', {'data_column' : 'Nature of Skillset', 'value' : ['sccm- networking', 'sccm-(network'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['java developer', '', 'Nature of Skillset', 'Nature of Skillset', 'string','java/developer', {'data_column' : 'Nature of Skillset', 'value' : ['java developer'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['java jsf', '', 'Nature of Skillset', 'Nature of Skillset', 'string','java/jsf', {'data_column' : 'Nature of Skillset', 'value' : ['java jsf'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['java tech lead', '', 'Nature of Skillset', 'Nature of Skillset', 'string','java/tech lead', {'data_column' : 'Nature of Skillset', 'value' : ['java tech lead'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['core java', '', 'Nature of Skillset', 'Nature of Skillset', 'string','java', {'data_column' : 'Nature of Skillset', 'value' : ['core java'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['analytical r & d', '', 'Nature of Skillset', 'Nature of Skillset', 'string','analytical r&d', {'data_column' : 'Nature of Skillset', 'value' : ['analytical r & d'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['automation testing java', '', 'Nature of Skillset', 'Nature of Skillset', 'string','automation testing/java', {'data_column' : 'Nature of Skillset', 'value' : ['automation testing java'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['cots developer', '', 'Nature of Skillset', 'Nature of Skillset', 'string','cots/developer', {'data_column' : 'Nature of Skillset', 'value' : ['cots developer'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['ra label', '', 'Nature of Skillset', 'Nature of Skillset', 'string','ra/label', {'data_column' : 'Nature of Skillset', 'value' : ['ra label'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['ra publishing', '', 'Nature of Skillset', 'Nature of Skillset', 'string','ra/publishing', {'data_column' : 'Nature of Skillset', 'value' : ['ra publishing'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['sccm-\(network, sharepoint,ms exchange\)', '', 'Nature of Skillset', 'Nature of Skillset', 'string','sccm-network, sharepoint,ms exchange', {'data_column' : 'Nature of Skillset', 'value' : ['sccm-(network, sharepoint,ms exchange)'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['tl|technical lead', '', 'Nature of Skillset', 'Nature of Skillset', 'string','tech lead', {'data_column' : 'Nature of Skillset', 'value' : ['tl','technical lead'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['it products and services|it services', '', 'Industry', 'Industry', 'string','it', {'data_column' : 'Industry', 'value' : ['it products and services','it services'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['standard chartered bank chennai', '', 'Client name', 'Client name', 'string','standard chartered bank', {'data_column' : 'Client name', 'value' : ['standard chartered bank chennai'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['hewitt|aon hewitt gurgaon', '', 'Client name', 'Client name', 'string','aon hewitt', {'data_column' : 'Client name', 'value' : ['hewitt','aon hewitt gurgaon'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['scheduled walk in|sceduled walkin', '', 'Interview Type', 'Interview Type', 'string','scheduled walkin', {'data_column' : 'Interview Type', 'value' : ['scheduled walk in','sceduled walkin'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['walkin ', '', 'Interview Type', 'Interview Type', 'string','walkin', {'data_column' : 'Interview Type', 'value' : ['walkin'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['t-24 developer', '', 'Nature of Skillset', 'Nature of Skillset', 'string','t 24/developer', {'data_column' : 'Nature of Skillset', 'value' : ['t-24 developer'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['chennai ', '', 'Location', 'Location', 'string','chennai', {'data_column' : 'Location', 'value' : ['chennai '], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['- cochin- ', '', 'Location', 'Location', 'string','cochin', {'data_column' : 'Location', 'value' : ['- cochin- '], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['gurgaonr', '', 'Location', 'Location', 'string','gurgaon', {'data_column' : 'Location', 'value' : ['gurgaonr'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['chennai ', '', 'Candidate Current Location', 'Candidate Current Location', 'string','chennai', {'data_column' : 'Candidate Current Location', 'value' : ['chennai '], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['- cochin- ', '', 'Candidate Current Location', 'Candidate Current Location', 'string','cochin', {'data_column' : 'Candidate Current Location', 'value' : ['- cochin- '], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['gurgaonr', '', 'Candidate Current Location', 'Candidate Current Location', 'string','gurgaon', {'data_column' : 'Candidate Current Location', 'value' : ['gurgaonr'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['chennai ', '', 'Interview Venue', 'Interview Venue', 'string','chennai', {'data_column' : 'Interview Venue', 'value' : ['chennai '], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['- cochin- ', '', 'Interview Venue', 'Interview Venue', 'string','cochin', {'data_column' : 'Interview Venue', 'value' : ['- cochin- '], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['gurgaonr', '', 'Interview Venue', 'Interview Venue', 'string','gurgaon', {'data_column' : 'Interview Venue', 'value' : ['gurgaonr'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['chennai ', '', 'Candidate Job Location', 'Candidate Job Location', 'string','chennai', {'data_column' : 'Candidate Job Location', 'value' : ['chennai '], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['- cochin- ', '', 'Candidate Job Location', 'Candidate Job Location', 'string','cochin', {'data_column' : 'Candidate Job Location', 'value' : ['- cochin- '], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['gurgaonr', '', 'Candidate Job Location', 'Candidate Job Location', 'string','gurgaon', {'data_column' : 'Candidate Job Location', 'value' : ['gurgaonr'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['chennai ', '', 'Candidate Native location', 'Candidate Native location', 'string','chennai', {'data_column' : 'Candidate Native location', 'value' : ['chennai '], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['- cochin- ', '', 'Candidate Native location', 'Candidate Native location', 'string','cochin', {'data_column' : 'Candidate Native location', 'value' : ['- cochin- '], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['gurgaonr', '', 'Candidate Native location', 'Candidate Native location', 'string','gurgaon', {'data_column' : 'Candidate Native location', 'value' : ['gurgaonr'], 'condition_func': (lambda x,y: x.isin(y))}]
diccionario.loc[len(diccionario)] = ['delhi /ncr', '', 'Candidate Native location', 'Candidate Native location', 'string','delhi', {'data_column' : 'Candidate Native location', 'value' : ['delhi /ncr'], 'condition_func': (lambda x,y: x.isin(y))}]

In [14]:
def parse(string,pattern):
    matches = re.search(pattern["lookup_regex"], string)
    parse_data = None
    if matches != None:
        if (pattern["value_type"] == 'float') and (pattern["extract_regex"] != ''):
            parse_data = float( re.search(pattern["extract_regex"], matches.group(0)).group(0).replace(".","") ) + float(pattern["value_offset"])
        elif (pattern["value_type"] == 'int') and (pattern["extract_regex"] != ''):
            parse_data = int( re.search(pattern["extract_regex"], matches.group(0)).group(0).replace(".","") ) + int(pattern["value_offset"])
        elif (pattern["value_type"] == 'string') and (pattern["extract_regex"] != ''):
            parse_data = re.search(pattern["extract_regex"], matches.group(0)).group(0) + pattern["value_offset"]
        elif pattern["value_type"] == 'float':
            parse_data = float(pattern["value_offset"])
        elif pattern["value_type"] == 'int':
            parse_data = int(pattern["value_offset"])
        else:
            parse_data = pattern["value_offset"]
    else:
        parse_data = np.nan;
    return parse_data

def df_parser(data,parse_dictionary):
    for pattern_row_index, pattern in parse_dictionary.iterrows():
        #rows_to_parse = (data[pattern["column_to_complete"]].isnull() == True) & (pattern['extra_condition']['condition_func'] (data[pattern['extra_condition']['data_column']],pattern['extra_condition']['value']) == True) 
        rows_to_parse = pattern['extra_condition']['condition_func'] (data[pattern['extra_condition']['data_column']],pattern['extra_condition']['value']) == True 
        info_to_parse = data[rows_to_parse][pattern["column_to_lookup"]]
        valid_info = info_to_parse[info_to_parse.apply(lambda a : (type(a) is str) and (a != ''))]
        data.loc[rows_to_parse, pattern["column_to_complete"]] = valid_info.apply(parse, args=([pattern]))

In [15]:
df_parser(df,diccionario)

In [16]:
df['Nature of Skillset'].value_counts().sort_index()

- sapbo, informatica                     4
accounting operations                   86
als testing                             15
aml/kyc/cdd                             84
analytical r&d                          16
automation testing/java                  7
banking operations                      24
basesas program/ reporting               1
biosimillar                             10
cdd kyc                                 52
cots                                     4
cots/developer                          13
dot net                                  9
emea                                     6
etl                                      9
fresher                                 86
generic drugs – ra                       4
global labelling                         6
hadoop                                  12
java                                    38
java                                    10
java, j2ee                               2
java, spring, hibernate                  2
java, sql  

#### Definimos una columna que cuenta la cantidad de skills de cada candidato

In [17]:
df['n_skills'] = df['Nature of Skillset'].apply(len)

In [18]:
set(df["Candidate Current Location"])

{'bangalore', 'chennai', 'cochin', 'delhi', 'gurgaon', 'hyderabad', 'noida'}

In [19]:
df["Industry"].value_counts()

bfsi               942
pharmaceuticals    165
it                  79
electronics         23
telecom             17
Name: Industry, dtype: int64

In [20]:
df["Client name"].value_counts()

standard chartered bank    914
hospira                     75
pfizer                      75
aon hewitt                  50
flextronics                 23
anz                         22
ust                         18
prodapt                     17
astrazeneca                 15
williams lea                11
barclays                     5
woori bank                   1
Name: Client name, dtype: int64

In [21]:
df["Have you obtained the necessary permission to start at the required time"].value_counts()

yes        917
no          84
not yet     19
Name: Have you obtained the necessary permission to start at the required time, dtype: int64

In [22]:
relevant_cols = ["Position to be closed"]
df = df.dropna(subset=relevant_cols)

In [23]:
df["Position to be closed"].value_counts()

routine                1016
niche                   163
dot net                  18
trade finance            11
aml                       8
production- sterile       5
selenium testing          5
Name: Position to be closed, dtype: int64

In [24]:
df["Interview Type"].unique()

array(['scheduled walkin', 'scheduled ', nan, 'walkin '], dtype=object)

In [25]:
print(set(df["Candidate Current Location"]))
print(set(df["Interview Venue"]))
print(set(df["Candidate Job Location"]))
print(set(df["Candidate Native location"]))

set(['cochin', 'hyderabad', 'gurgaon', 'chennai', 'noida', 'delhi', 'bangalore'])
set(['cochin', 'hyderabad', 'gurgaon', 'chennai', 'noida', 'hosur', 'bangalore'])
set(['cochin', 'gurgaon', 'chennai', 'noida', 'hosur', 'visakapatinam', 'bangalore'])
set(['hyderabad', 'kanpur', 'chitoor', 'ghaziabad', 'chandigarh', 'hissar', 'panjim', 'salem', 'visakapatinam', 'tanjore', 'agra', 'cuttack', 'pondicherry', 'tirupati', 'mumbai', 'warangal', 'trichy', 'ambur', 'hosur', 'patna', 'kolkata', 'nagercoil', 'coimbatore', 'gurgaon', 'chennai', 'noida', 'faizabad', 'baddi', 'delhi', 'bhubaneshwar', 'trivandrum', 'allahabad', 'tuticorin', 'kurnool', 'bangalore', 'ahmedabad', 'anantapur', 'cochin', 'pune', 'lucknow', 'belgaum', 'vellore', 'mysore', 'vijayawada'])


In [26]:
df["match_interview"] = df["Candidate Current Location"] == df["Interview Venue"]
df["match_job"] = df["Candidate Current Location"] == df["Candidate Job Location"]
df["match_nacim"] = df["Candidate Job Location"] == df["Candidate Native location"]

## One Hot Encoding

In [27]:
onehot_skills = df["Nature of Skillset"].str.split(" , | ,|, |,| /|/ |/| - | -|- |-| – ", expand=True).stack().str.get_dummies().sum(level=0)
onehot_skills

,accounting operations,als testing,aml,analytical r&d,automation testing,banking operations,basesas program,biosimillar,cdd,cdd kyc,...,sharepoint,spring,sql,sr automation testing,struts,submission management,t 24,tech lead,testing,xml
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
onehot_skills.describe()

,accounting operations,als testing,aml,analytical r&d,automation testing,banking operations,basesas program,biosimillar,cdd,cdd kyc,...,sharepoint,spring,sql,sr automation testing,struts,submission management,t 24,tech lead,testing,xml
count,1226.000000,1226.000000,1226.000000,1226.000000,1226.000000,1226.000000,1226.000000,1226.000000,1226.000000,1226.000000,...,1226.000000,1226.000000,1226.000000,1226.000000,1226.000000,1226.000000,1226.000000,1226.000000,1226.000000,1226.000000
mean,0.070147,0.012235,0.068515,0.013051,0.005710,0.019576,0.000816,0.008157,0.068515,0.042414,...,0.001631,0.038336,0.009788,0.010604,0.181892,0.001631,0.012235,0.015498,0.008972,0.002447
std,0.255499,0.109978,0.252732,0.113538,0.075377,0.138594,0.028560,0.089982,0.252732,0.201615,...,0.040373,0.192085,0.098489,0.102468,0.385913,0.040373,0.109978,0.123571,0.094335,0.049427
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [29]:
df.drop(["Nature of Skillset"], axis=1, inplace=True)
df = pd.concat([df, onehot_skills], axis=1, sort=False)

In [30]:
onehot_client = pd.get_dummies(df["Client name"],prefix="cli")
df.drop(["Client name"], axis=1, inplace=True)
df = pd.concat([df, onehot_client], axis=1, sort=False)

In [31]:
onehot_industry = pd.get_dummies(df["Industry"],prefix="ind")
df.drop(["Industry"], axis=1, inplace=True)
df = pd.concat([df, onehot_industry], axis=1, sort=False)

In [32]:
onehot_gender = pd.get_dummies(df["Gender"])
df.drop(["Gender"], axis=1, inplace=True)
df = pd.concat([df, onehot_gender], axis=1, sort=False)

In [33]:
onehot_position = pd.get_dummies(df["Position to be closed"],prefix="pos")
df.drop(["Position to be closed"], axis=1, inplace=True)
df = pd.concat([df, onehot_position], axis=1, sort=False)

In [34]:
onehot_location = pd.get_dummies(df["Location"],prefix="loc")
df.drop(["Location"], axis=1, inplace=True)
df = pd.concat([df, onehot_location], axis=1, sort=False)

In [35]:
onehot_interview_type = pd.get_dummies(df["Interview Type"],prefix="typ")
df.drop(["Interview Type"], axis=1, inplace=True)
df = pd.concat([df, onehot_interview_type], axis=1, sort=False)

In [36]:
onehot_question_a = pd.get_dummies(df["Have you obtained the necessary permission to start at the required time"],prefix="qa")
df.drop(["Have you obtained the necessary permission to start at the required time"], axis=1, inplace=True)
df = pd.concat([df, onehot_question_a], axis=1, sort=False)

In [37]:
onehot_question_b = pd.get_dummies(df["Hope there will be no unscheduled meetings"],prefix="qb")
df.drop(["Hope there will be no unscheduled meetings"], axis=1, inplace=True)
df = pd.concat([df, onehot_question_b], axis=1, sort=False)

In [38]:
onehot_question_c = pd.get_dummies(df["Can I Call you three hours before the interview and follow up on your attendance for the interview"],prefix="qc")
df.drop(["Can I Call you three hours before the interview and follow up on your attendance for the interview"], axis=1, inplace=True)
df = pd.concat([df, onehot_question_c], axis=1, sort=False)

In [39]:
onehot_question_d = pd.get_dummies(df["Can I have an alternative number/ desk number. I assure you that I will not trouble you too much"],prefix="qd")
df.drop(["Can I have an alternative number/ desk number. I assure you that I will not trouble you too much"], axis=1, inplace=True)
df = pd.concat([df, onehot_question_d], axis=1, sort=False)

In [40]:
onehot_question_e = pd.get_dummies(df["Have you taken a printout of your updated resume. Have you read the JD and understood the same"],prefix="qe")
df.drop(["Have you taken a printout of your updated resume. Have you read the JD and understood the same"], axis=1, inplace=True)
df = pd.concat([df, onehot_question_e], axis=1, sort=False)

In [41]:
onehot_question_f = pd.get_dummies(df["Are you clear with the venue details and the landmark."],prefix="qf")
df.drop(["Are you clear with the venue details and the landmark."], axis=1, inplace=True)
df = pd.concat([df, onehot_question_f], axis=1, sort=False)

In [42]:
onehot_question_g = pd.get_dummies(df["Has the call letter been shared"],prefix="qg")
df.drop(["Has the call letter been shared"], axis=1, inplace=True)
df = pd.concat([df, onehot_question_g], axis=1, sort=False)

In [43]:
onehot_ccl = pd.get_dummies(df["Candidate Current Location"],prefix="ccl")
df.drop(["Candidate Current Location"], axis=1, inplace=True)
df = pd.concat([df, onehot_ccl], axis=1, sort=False)

In [44]:
onehot_cjl = pd.get_dummies(df["Candidate Job Location"],prefix="cjl")
df.drop(["Candidate Job Location"], axis=1, inplace=True)
df = pd.concat([df, onehot_cjl], axis=1, sort=False)

In [45]:
onehot_iv = pd.get_dummies(df["Interview Venue"],prefix="iv")
df.drop(["Interview Venue"], axis=1, inplace=True)
df = pd.concat([df, onehot_iv], axis=1, sort=False)

In [46]:
onehot_cnl = pd.get_dummies(df["Candidate Native location"],prefix="cnl")
df.drop(["Candidate Native location"], axis=1, inplace=True)
df = pd.concat([df, onehot_cnl], axis=1, sort=False)

In [47]:
onehot_ms = pd.get_dummies(df["Marital Status"],prefix="ms")
df.drop(["Marital Status"], axis=1, inplace=True)
df = pd.concat([df, onehot_ms], axis=1, sort=False)

In [48]:
print(df.columns)

Index([u'Observed Attendance', u'n_skills', u'match_interview', u'match_job',
       u'match_nacim', u'accounting operations', u'als testing', u'aml',
       u'analytical r&d', u'automation testing',
       ...
       u'cnl_tirupati', u'cnl_trichy', u'cnl_trivandrum', u'cnl_tuticorin',
       u'cnl_vellore', u'cnl_vijayawada', u'cnl_visakapatinam',
       u'cnl_warangal', u'ms_married', u'ms_single'],
      dtype='object', length=197)


### Preparando la variable target para clasificar:

In [49]:
set(df['Observed Attendance'])

{'no', 'no ', 'yes', 'yes '}

In [50]:
df['Observed Attendance'] = df['Observed Attendance'].replace(['no', 'no ', 'yes', 'yes '],[0,0,1,1])
df['Observed Attendance'].value_counts()

1    778
0    448
Name: Observed Attendance, dtype: int64